In [23]:
import os
import cv2
import numpy as np
import packcircles as pc
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [24]:
plt.rcParams["figure.figsize"] = (14, 14)
max_radius = 0.2
img_path = "./images/"
max_vel = 0.5

In [25]:
def is_close(a, b, eps):
    a = np.array(a)
    b = np.array(b)
    return np.linalg.norm(a-b)<=eps

In [26]:
class Particle:
    
    def __init__(self, coor, vel, rad, m):
        self.coordinate = coor
        self.velocity = vel
        self.radius = rad
        self.mass = m
        self.color = np.random.rand(3,)
    
    def motion_update(self, acceleration, delta_t):
        self.coordinate += self.velocity * delta_t
        self.velocity += acceleration * delta_t

In [27]:
class Environment:
    
    def __init__(self, d_max, dt):
        self.D_max = d_max
        self.particles = []
        self.delta_t = dt
        
    def generate_particles(self, N):
        print("Generating Particles...")
        radii = [max_radius for _ in range(N)]
        circles = pc.pack(radii)
        for (x, y, radius) in circles:
            mass = 1
            temp_coor = np.array([x, y]) + np.array([self.D_max/2, self.D_max/2])
            temp_vel = np.random.uniform(-1, 1, (2,))*max_vel
            particle = Particle(temp_coor, temp_vel, radius*0.8, mass)
            self.particles.append(particle)

    def show_environment(self, i):
        fig, ax = plt.subplots()
        ax.set_xlim([0, self.D_max])
        ax.set_ylim([0, self.D_max])
        energy = 0
        momentum = np.zeros(2,)
        for p in self.particles:
            momentum += p.mass * p.velocity
            energy += 0.5 * p.mass * np.dot(p.velocity, p.velocity)
            circle = plt.Circle(p.coordinate, p.radius, color=p.color)
            ax.add_patch(circle)
        ax.set_title(f'Energy: {energy} & Momentum: {momentum}')
        fig.savefig(f"./images/{i}.jpg")
        plt.close(fig)
    
    def fix_border(self, particle):
        x, y = particle.coordinate
        if is_close((x), (self.D_max), particle.radius) or is_close(x, 0, particle.radius): 
            particle.velocity = np.multiply(particle.velocity, np.array([-1, 1]))
            particle.just_border = True
        if is_close((y), (self.D_max), particle.radius) or is_close(y, 0, particle.radius): 
            particle.velocity = np.multiply(particle.velocity, np.array([1, -1]))
            particle.just_border =  True
        return particle
    
    def check_collisions(self):
        for i in range(len(self.particles)-1):
            for j in range(i+1, len(self.particles)):
                p1 = self.particles[i]
                p2 = self.particles[j]
                x1 = p1.coordinate
                x2 = p2.coordinate
                dist = p1.radius + p2.radius
                if is_close(x1, x2, dist):
                    m1 = p1.mass
                    m2 = p2.mass
                    v1 = p1.velocity
                    v2 = p2.velocity
                    dist = np.linalg.norm(x1-x2)
                    self.particles[i].velocity = v1 - (2*m2/(m1+m2)) * np.dot(v1-v2, x1-x2) * (1/(dist**2)) * (x1 - x2)
                    self.particles[j].velocity = v2 - (2*m1/(m1+m2)) * np.dot(v2-v1, x2-x1) * (1/(dist**2)) * (x2 - x1)
                    
    
    def update_environment(self):
        for p in self.particles: 
            p.motion_update(0, self.delta_t)
            p = self.fix_border(p)
        
        self.check_collisions()
        return
            

In [28]:
fps = 120
duration = 10

env = Environment(10, 1/fps)
env.generate_particles(30)
i = 0
print("Running Simulation...")
for i in tqdm(range(fps*duration)):
    env.update_environment()
    env.show_environment(i)
    i+=1

Generating Particles...
Running Simulation...


  0%|          | 0/1200 [00:00<?, ?it/s]

In [29]:
img_array = []
files = os.listdir(img_path)
files = sorted(files, key=lambda x: int(x.split('.')[0]))

print("Reading Images...")

for filename in tqdm(files):
    img = cv2.imread(img_path+filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)


out = cv2.VideoWriter('project.avi',cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

print("Making The video...") 
for i in tqdm(range(len(img_array))):
    out.write(img_array[i])
out.release()

Reading Images...


  0%|          | 0/1200 [00:00<?, ?it/s]

Making The video...


  0%|          | 0/1200 [00:00<?, ?it/s]